In [1]:
# Установите необходимые библиотеки
!pip install transformers datasets peft accelerate evaluate tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 46.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 64.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 57.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 72.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 156.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 76.4 MB/s eta 0

In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from tqdm import tqdm
import torch
import evaluate
import pandas as pd

/home/ubuntu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. Загрузка и подготовка данных
def preprocess_data(example):
    """Форматируем данные для задачи QA."""
    input_text = f"question: {example['question']} context: {example['context']}"
    target_text = example['answers']['text'][0] if example['answers']['text'] else ""
    return {'input_text': input_text, 'target_text': target_text}

# Загрузка данных
dataset = load_dataset("squad")
train_data = dataset['train'].map(preprocess_data, remove_columns=dataset['train'].column_names)
validation_data = dataset['validation'].map(preprocess_data, remove_columns=dataset['validation'].column_names)

In [4]:
# Преобразование данных в токены
def tokenize_data(batch, tokenizer):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=512)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b", use_fast=False)
train_data = train_data.map(lambda x: tokenize_data(x, tokenizer), batched=True)
validation_data = validation_data.map(lambda x: tokenize_data(x, tokenizer), batched=True)

In [5]:
# Удаляем ненужные колонки и задаем формат
train_data = train_data.remove_columns(["input_text", "target_text"])
validation_data = validation_data.remove_columns(["input_text", "target_text"])
train_data.set_format("torch")
validation_data.set_format("torch")

In [6]:
# 2. Функция для обучения и оценки модели
def train_and_evaluate(model, training_args, train_dataset, eval_dataset, tokenizer, config_name):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        processing_class=tokenizer,
    )
    trainer.train()

    # Оценка модели
    metric = evaluate.load("squad")
    results = trainer.evaluate()
    predictions = trainer.predict(eval_dataset)
    decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)
    formatted_preds = [{"id": str(i), "prediction_text": pred} for i, pred in enumerate(decoded_preds)]
    formatted_labels = [{"id": str(i), "answers": {"text": [label]}} for i, label in enumerate(decoded_labels)]
    metrics = metric.compute(predictions=formatted_preds, references=formatted_labels)
    metrics["Config"] = config_name
    return metrics

In [7]:
# Функция для оценки базовых моделей
def evaluate_baseline(model, eval_dataset, tokenizer, config_name):
    metric = evaluate.load("squad")
    model.eval()

    predictions = []
    references = []

    # Генерация предсказаний
    for i in tqdm(range(len(eval_dataset))):
        input_ids = eval_dataset[i]["input_ids"].unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
        # Using max_new_tokens instead of max_length
        output_ids = model.generate(input_ids, max_new_tokens=128, num_beams=4)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        reference = tokenizer.decode(eval_dataset[i]["labels"], skip_special_tokens=True)
        predictions.append({"id": str(i), "prediction_text": prediction})
        references.append({"id": str(i), "answers": {"text": [reference]}})

    # Вычисление метрик
    metrics = metric.compute(predictions=predictions, references=references)
    metrics["Config"] = config_name
    return metrics

In [8]:
# 3. Настройки для разных экспериментов
results = []

In [ ]:
# a) Baseline: Модель без дообучения
for model_name in ["facebook/opt-1.3b", "facebook/opt-2.7b", "facebook/opt-6.7b"]:
    model_baseline = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
    metrics = evaluate_baseline(model_baseline, validation_data, tokenizer, f"Baseline ({model_name})")
    results.append(metrics)

In [ ]:
# b) Fine-tuning: Полная настройка модели
for model_name in ["facebook/opt-1.3b", "facebook/opt-2.7b", "facebook/opt-6.7b"]:
    model_finetune = AutoModelForCausalLM.from_pretrained(model_name)
    training_args_finetune = TrainingArguments(
        output_dir=f"./results_finetune_{model_name.split('/')[-1]}",
        per_device_train_batch_size=4,
        num_train_epochs=3,
        eval_strategy="steps",
        save_steps=1000,
        logging_steps=10,
        learning_rate=2e-4,
        fp16=True,
    )
    metrics = train_and_evaluate(
        model_finetune, training_args_finetune, train_data, validation_data, tokenizer, f"Full Fine-tuning ({model_name})"
    )
    results.append(metrics)

In [ ]:
# c) LoRA: Настройка с разными значениями ранга r
for r in [4, 8, 16]:
    for model_name in ["facebook/opt-1.3b", "facebook/opt-2.7b", "facebook/opt-6.7b"]:
        model_lora = AutoModelForCausalLM.from_pretrained(model_name)
        lora_config = LoraConfig(
            r=r,
            lora_alpha=32,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=0.1,
            bias="none"
        )
        lora_model = get_peft_model(model_lora, lora_config)
        training_args_lora = TrainingArguments(
            output_dir=f"./results_{model_name.split('/')[-1]}_lora_r{r}",
            per_device_train_batch_size=8,
            num_train_epochs=3,
            eval_strategy="steps",
            save_steps=1000,
            logging_steps=10,
            learning_rate=2e-4,
            fp16=True,
        )
        metrics = train_and_evaluate(lora_model, training_args_lora, train_data, validation_data, tokenizer, f"{model_name} LoRA (r={r})")
        results.append(metrics)

/home/ubuntu/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/tmp/ipykernel_2515/2118481475.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 4. Вывод результатов
results_df = pd.DataFrame(results)
print(results_df)
results_df.to_csv("results_comparison.csv", index=False)